In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bug-data/bugs-data.csv


In [27]:
import regex as re
from tqdm import tqdm
tqdm.pandas()

from gensim.models import Word2Vec
import texthero as hero
from textblob import TextBlob

from gensim.models import FastText


In [3]:
data = pd.read_csv('../input/bug-data/bugs-data.csv')
data.head()

,bugid,summary,component
0,808252,home and end key not working on web pages open...,Keyboard Navigation
1,1389748,[Linux] default browser firefox not working,Shell Integration
2,1588476,Full page scrolling screenshot not working for...,PDF Viewer
3,1609740,"Redhat Linux EL 6.10 Firefox v68, Set as Deskt...",Shell Integration
4,1617503,Bookmark button not working after system retur...,Bookmarks & History


In [5]:
!pip install texthero

     |████████████████████████████████| 24.2 MB 6.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 21.2 MB/s eta 0:00:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
  Attempting uninstall: gensim
    Found existing installation: gensim 4.0.1
    Uninstalling gensim-4.0.1:
      Successfully uninstalled gensim-4.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scattertext 0.1.3 requires gensim>=4.0.0, but you have gensim 3.8.3 which is incompatible.
pyldavis 3.3.1 requires numpy>=1.20.0, but you have numpy 1.19.5 which is incompatible.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.6.2 which is incompatible.


In [6]:
data["summary"] = hero.clean(data["summary"])
data.head()

,bugid,summary,component
0,808252,home end key working web pages opened firefox ...,Keyboard Navigation
1,1389748,linux default browser firefox working,Shell Integration
2,1588476,full page scrolling screenshot working pdf,PDF Viewer
3,1609740,redhat linux el firefox v68 set desktop backgr...,Shell Integration
4,1617503,bookmark button working system returning standby,Bookmarks & History


In [12]:
def sent_lemmatize(sentence):
    sent = TextBlob(sentence)
    return (" ". join([w.lemmatize() for w in sent.words]))
    
data["summary"] = data["summary"].progress_apply(lambda x: sent_lemmatize(x))


100%|██████████| 2764/2764 [00:00<00:00, 3496.91it/s]


In [15]:
NUM_TOP_WORDS = 20
hero.top_words(data["summary"])[:NUM_TOP_WORDS]

working      2791
firefox       657
stop          317
tab           262
button        208
window        205
page          202
stopped       172
available     142
extension     142
bookmark      135
search        134
properly      133
menu          124
bar           122
link          116
browser       110
new           109
update         98
click          90
Name: summary, dtype: int64

In [17]:
sentences = data['summary'].progress_apply(lambda x: x.split())
sentences = list(sentences)

100%|██████████| 2764/2764 [00:00<00:00, 286743.91it/s]


## WORD2VEC MODEL

In [37]:
n = 50
model = Word2Vec(sentences, window = 8)
model.train(sentences,total_examples=len(sentences), epochs=10)

(89244, 176740)

In [44]:
list_of_words_w2v = list(model.wv.vocab)
print(f"Total number of words in trained word2vec : {len(list_of_words_w2v)}")
print(list_of_words_w2v[:50])

Total number of words in trained word2vec : 548
['home', 'end', 'key', 'working', 'web', 'page', 'opened', 'firefox', 'browser', 'linux', 'default', 'full', 'scrolling', 'screenshot', 'pdf', 'set', 'desktop', 'background', 'dialog', 'appears', 'image', 'bookmark', 'button', 'system', 'new', 'tab', 'stop', 'link', 'closing', 'url', 'auto', 'screen', 'homepage', 'shortcut', 'multiple', 'open', 'sometimes', 'click', 'close', 'need', 'closed', 'bar', 'start', 'keyboard', 'alt', 'select', 'location', 'suddenly', 'arrow', 'focus']


In [26]:
#Let's see how it worked
word_test = 'search'
print('Words similar to {} are: '.format(word_test))
print(model.wv.most_similar(positive=word_test))


Words similar to search are: 
[('url', 0.9990907907485962), ('engine', 0.999040961265564), ('autocomplete', 0.9990193843841553), ('arrow', 0.9988964200019836), ('end', 0.9988788366317749), ('suggestion', 0.9988028407096863), ('web', 0.9987673163414001), ('input', 0.9987359046936035), ('navigation', 0.9987213611602783), ('home', 0.9986757040023804)]


In [48]:
# Visualization of word2Vec embedded words in 2D using T-SNE transform  
from sklearn.decomposition import PCA, TruncatedSVD, SparsePCA
from sklearn.manifold import TSNE

import plotly.express as px
import plotly.graph_objs as go
from plotly import tools

tsne = TSNE(n_components=2, random_state=0)
w2v_data = model.wv[model.wv.vocab]
w2v_tsne_transformed = tsne.fit_transform(w2v_data[:100])

# create a scatter plot for projecting glove vetors in 2D

fig = px.scatter(x=w2v_tsne_transformed[:,0], y=w2v_tsne_transformed[:,1],text=list_of_words_w2v[:100])
fig.update_traces(textposition='bottom center')
fig.update_layout(
     width=1100,
    height=900,
    title_text='Visualization of Word2Vec embedded words in 2D using T-SNE transform'
)
fig.show()

## FAST TEXT MODEL

In [28]:
ft_model = FastText(sentences, size=n, window=8, min_count=5, workers=2,sg=1)

In [29]:
word_test = 'search'
print('Words similar to {} are: '.format(word_test))
print(ft_model.wv.most_similar(positive=word_test))


Words similar to search are: 
[('suggestion', 0.9997535943984985), ('autocomplete', 0.9997377991676331), ('completely', 0.9997295141220093), ('navigation', 0.9997268915176392), ('toolbar', 0.9997125864028931), ('desktop', 0.9997125864028931), ('complete', 0.9997114539146423), ('properly', 0.9997090697288513), ('several', 0.9997081756591797), ('protection', 0.9997081160545349)]
